In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets # for the widgets
from IPython.display import display # to display them
import pylab
from sklearn.compose import ColumnTransformer
import sklearn

Mounted at /content/drive


In [4]:
start_clip_value = 100
def View_results(value=0,end_value=0):
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    ax.set_xlabel('Time')
    ax.set_ylabel('Value of J')
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Value of all variables')
    ax.set_title("Prediction vs True value")
    ax.plot(fin_values[value][start_clip_value:end_value],c='red',label='prediction')
    ax.plot(original_vals[value][start_clip_value:end_value],c='green',label='Groundtruth')
    ax.legend()
    ax2.set_title("SIR model")
    plt.plot(batches_of_TestX[value][start_clip_value:end_value])
    plt.show()
widgets.interact(View_results,value=widgets.IntSlider(
    value=0,
    min=0,
    max=no_of_sims_for_testing,
    step=1,
    description='TestSimNumber'),end_value=widgets.IntSlider(
    value=5000,
    min=0,
    max=timesteps,
    step=1,
    description='X_lim_of_simulation'));

interactive(children=(IntSlider(value=0, description='TestSimNumber'), IntSlider(value=5000, description='X_li…

1. **Time delta: 1/1000th timeunit** (10 seconds, 10,000 timesteps)
2. Red line is entirely depicting predictions
3. For every single prediction (~10,000 in all), it takes values of [T], [I], [V], [J] from (t-1) ,(t-2) , (t-3) as input (4 * 3 = 12 inputs, one output)
4. beta is unknown to the model and varies across 25% of original range

In [ ]:
dimension_to_predict = 1
no_of_epochs = 200
loadedArr = np.loadtxt('drive/My Drive/Colab Notebooks/S_1_point_3.csv')
loadedOriginal = loadedArr.reshape(loadedArr.shape[0], loadedArr.shape[1] // 10, 10)
a = loadedOriginal
simulations = a.shape[0]
timesteps = a.shape[1]
dimensions = a.shape[2]
no_of_sims_for_testing = 100
a_test = a[:no_of_sims_for_testing]
a_train = a[no_of_sims_for_testing:]
batches_of_TrainX = []
for i in range(a_train.shape[0]):
        data = {
        'time_steps' : a_train[i,:,0], 
        '[T]' : a_train[i,:,1], 
        #'[I]' : a_train[i,:,2],
        #'[V]' : a_train[i,:,3],
        #'[J]' : a_train[i,:,4] ,
               }
        data = pd.DataFrame(data) 
        batches_of_TrainX.append(data)
batches_of_TrainX = np.array(batches_of_TrainX)
scaled_batches = batches_of_TrainX.reshape(batches_of_TrainX.shape[0]*batches_of_TrainX.shape[1],batches_of_TrainX.shape[2])
scaler_1 = sklearn.preprocessing.MinMaxScaler()
#sklearn.preprocessing.MinMaxScaler()
#sklearn.preprocessing.StandardScaler()
scaler_1 = scaler_1.fit(scaled_batches)
scaler_2 = sklearn.preprocessing.MinMaxScaler()
scaler_2 = scaler_2.fit(scaled_batches[:,dimension_to_predict].reshape(-1,1))
scaler_3 = sklearn.preprocessing.MinMaxScaler()
scaler_3 = scaler_1.fit(scaled_batches[:,:5])
scaled_batches = scaler_1.transform(scaled_batches)
trainX = []
for i in range(batches_of_TrainX.shape[0]):
    trainX.append(scaled_batches[i*batches_of_TrainX.shape[1]:(i+1)*batches_of_TrainX.shape[1]])  
trainX = np.array(trainX)
batches_of_TrainX_array = trainX
TrainX = []
TrainY = []
n_future = 1  
n_past = 10 
window_end_number = timesteps - n_past
for i in range(batches_of_TrainX_array.shape[0]):
    train_batch = []
    train_tempY = []
    for j in range(n_past, batches_of_TrainX_array.shape[1] - n_future +1):
        train_batch.append(batches_of_TrainX_array[i,j-n_past:j,:])
        train_tempY.append(batches_of_TrainX_array[i, j+n_future-1 : j+n_future , dimension_to_predict])
    TrainX.append(train_batch)
    TrainY.append(train_tempY)
TrainX = np.array(TrainX)
TrainY = np.array(TrainY)

opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, clipvalue=0.5)
model = Sequential()
model.add(LSTM(32, batch_input_shape=(TrainX.shape[1],TrainX.shape[2], TrainX.shape[3]), return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(32, return_sequences=False))
model.add(Dense(TrainY.shape[2]))
model.compile(optimizer='Adam', loss='mse')

TrainX = TrainX.reshape(TrainX.shape[0]*TrainX.shape[1],TrainX.shape[2],TrainX.shape[3])
TrainY = TrainY.reshape(TrainY.shape[0]*TrainY.shape[1],TrainY.shape[2])
#model = tf.keras.models.load_model('drive/My Drive/Colab Notebooks/pred_V_logless_oneinput')
model = tf.keras.models.load_model('drive/My Drive/Colab Notebooks/pred_T_logless_oneinput')
#model = tf.keras.models.load_model('drive/My Drive/Colab Notebooks/pred_I_logless_oneinput')
#model = tf.keras.models.load_model('drive/My Drive/Colab Notebooks/pred_J_logless_oneinput')

batches_of_TestX = []
for i in range(a_test.shape[0]):
        data = {
        'time_steps' : a_test[i,:,0], 
        '[T]' : a_test[i,:,1], 
        #'[I]' : a_test[i,:,2],
        #'[V]' : a_test[i,:,3],
        #'[J]' : a_test[i,:,4] ,
               }
        data = pd.DataFrame(data) 
        batches_of_TestX.append(data)
batches_of_TestX = np.array(batches_of_TestX)
scaled_batches_1 = batches_of_TestX.reshape(batches_of_TestX.shape[0]*batches_of_TestX.shape[1],batches_of_TestX.shape[2])
scaled_batches_1 = scaler_1.transform(scaled_batches_1)
testX = []
for i in range(batches_of_TestX.shape[0]):
    testX.append(scaled_batches[i*batches_of_TestX.shape[1]:(i+1)*batches_of_TestX.shape[1]])  
testX = np.array(testX)
TestX = []
for i in range(testX.shape[0]):
    test_batch = []
    for j in range(n_past, testX.shape[1] - n_future +1):
        test_batch.append(testX[i,j-n_past:j,:])
    TestX.append(test_batch)
TestX = np.array(TestX)
TestX = []
for i in range(testX.shape[0]):
    test_batch = []
    for j in range(n_past, testX.shape[1] - n_future +1):
        test_batch.append(testX[i,j-n_past:j,:])
    TestX.append(test_batch)
TestX = np.array(TestX)
TestX = TestX.reshape(TestX.shape[0]*TestX.shape[1],TestX.shape[2],TestX.shape[3])
Y_pred_V = model.predict(TestX,batch_size=window_end_number)
Y_pred_V = Y_pred_V.reshape(-1)
Y_pred_V_scaled_back = scaler_2.inverse_transform(Y_pred_V.reshape(-1,1))
fin_values = []
for i in range(100):
    fin_values.append(Y_pred_V_scaled_back[i*window_end_number:(i+1)*window_end_number])  
fin_values = np.array(fin_values)
fin_values = fin_values.reshape(100,window_end_number)
original_vals = a_test[:,:,:]
original_vals.shape
original_vals = original_vals[:,timesteps-window_end_number:,1]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (9990, 10, 32)            4480      
                                                                 
 lstm_1 (LSTM)               (9990, 10, 64)            24832     
                                                                 
 lstm_2 (LSTM)               (9990, 10, 128)           98816     
                                                                 
 lstm_3 (LSTM)               (9990, 32)                20608     
                                                                 
 dense (Dense)               (9990, 1)                 33        
                                                                 
Total params: 148,769
Trainable params: 148,769
Non-trainable params: 0
_________________________________________________________________
